# MobiML Transforms Demo

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import pickle
import holoviews as hv
from hvplot import pandas
from holoviews import opts
from holoviews.element import tiles
from datetime import datetime, timedelta
from sklearn.metrics import log_loss

import sys
sys.path.append("..")

from mobiml.utils import convert_wgs_to_utm
from mobiml.datasets import AISDK, PreprocessedAISDK, MOVER_ID
from mobiml.transforms import StationaryClientExtractor, TrajectoryAggregator, AISTripExtractor, AISLoader
from mobiml.models import SummarizedAISTrajectoryClassifier

opts.defaults(opts.Overlay(active_tools=['wheel_zoom']))
BG_TILES = tiles.CartoLight()

## Extracting AIS records around specific locations


In [ ]:
antennas = ['Point (11.96524 57.70730)', 'Point (11.63979 57.71941)', 'Point (11.78460 57.57255)']
antenna_radius_meters = 25000 
epsg_code = convert_wgs_to_utm(11.96524, 57.70730)

def create_client_gdf(clients, client_radius_meters) -> gpd.GeoDataFrame:
    ids =  [{'client': i} for i in range(len(clients))]
    df = pd.DataFrame(ids)
    df['geometry'] = gpd.GeoSeries.from_wkt(clients)
    gdf = gpd.GeoDataFrame(df, geometry=df.geometry, crs=4326)
    gdf = gdf.to_crs(epsg_code)
    gdf['geometry'] = gdf.buffer(client_radius_meters)
    return gdf.to_crs(4326)

buffered_antennas = create_client_gdf(antennas, antenna_radius_meters)
buffered_antennas.hvplot(tiles=BG_TILES, geo=True, alpha=0.5)

In [ ]:
min_lon, min_lat, max_lon, max_lat = buffered_antennas.geometry.total_bounds
aisdk = AISDK('./data/aisdk_20180208_sample.zip', min_lon, min_lat, max_lon, max_lat)
aisdk.datashade(width=700, height=400)

In [ ]:
print(f"{datetime.now()} Extracting client data ...")
client_gdf = StationaryClientExtractor(aisdk, buffered_antennas) 

In [ ]:
out_path = './data/ais-extracted-stationary.feather'
print(f"{datetime.now()} Writing output to {out_path}")
client_gdf.to_feather(out_path)

## Reading extracted AIS records from file

In [ ]:
aisdk = PreprocessedAISDK(out_path)
aisdk.df

In [ ]:
trajs = aisdk.to_trajs()
trajs

In [ ]:
aisdk.datashade(width=700, height=400)

In [ ]:
def plot_type(my_type):
    tmp = aisdk.copy()
    tmp.df = tmp.df[tmp.df.ship_type==my_type]
    return tmp.datashade(title=my_type, width=400, height=400)

plot_type("Cargo") + plot_type("Passenger") + plot_type("Tanker")

## Extract trips and compute features

In [ ]:
h3_resolution = 8

def create_vessel_list(gdf):
    print(f"{datetime.now()} Creating vessel list ...")
    return gdf.groupby(MOVER_ID)[['ship_type','Name']].agg(pd.Series.mode)


def pickle_vessels(vessels, out_path):
    print(f"{datetime.now()} Writing vessels to {out_path} ...")
    with open(out_path, 'wb') as out_file:
        pickle.dump(vessels, out_file)


def pickle_trajectories(trajs, out_path):
    print(f"{datetime.now()} Writing trajectories to {out_path} ...")
    with open(out_path, 'wb') as out_file:
        pickle.dump(trajs, out_file)
    return out_path

In [ ]:
print(f"{datetime.now()} Loading data from {out_path} ...")
gdf =  gpd.read_feather(out_path)
vessels = create_vessel_list(gdf)

In [ ]:
print(f"{datetime.now()} Extracting trips ...")
trajs = AISTripExtractor(gdf).get_trips(gap_duration=timedelta(minutes=60))  # create_trajs(gdf)

In [ ]:
print(f"{datetime.now()} Computing trajectory features ...")
trajs = TrajectoryAggregator(trajs, vessels).aggregate_trajs(h3_resolution)   

In [ ]:
traj_path = 'data/ais-extracted-trajectories.pickle' 
pickle_trajectories(trajs, traj_path)
pickle_vessels(vessels, 'data/ais-extracted-vessels.pickle')

## Train & evaluate trajectory classifier

In [ ]:
vessel_types = ['Cargo', 'Passenger', 'Tanker']
features = ['speed_max', 'speed_median', 'x_start', 'y_start', 'x_end', 'y_end', 'length']

In [ ]:
stationary = AISLoader(vessel_types, features, 0.33, path='data/trajs-stationary.pickle')
(X_train, y_train), (X_test, y_test) = stationary.load()
print("Train: ", X_train.shape)
print("Test: ", X_test.shape)

In [ ]:
model = SummarizedAISTrajectoryClassifier(vessel_types, len(features))
model.fit(X_train, y_train)

In [ ]:
predicted = model.predict_proba(X_test)
predicted

In [ ]:
loss = log_loss(y_test, predicted, labels=vessel_types)
accuracy = model.score(X_test, y_test)
print("Accuracy", accuracy)
print("Loss", loss)